In [ ]:
import pandas as pdfrom sklearn.preprocessing import LabelEncoder, StandardScalerfrom sklearn.impute import SimpleImputerimport pickleimport osif __name__ == '__main__':    # Load the training data (for preprocessing)    train_data = pd.read_csv("../data/cell2celltrain.csv")    # Remove customer_id    train_data = train_data.drop('CustomerID', axis=1)    # Handle missing values BEFORE Label Encoding    for col in train_data.columns:        if train_data[col].dtype == 'object':            train_data[col] = train_data[col].fillna('Unknown')    # Convert categorical features to numerical features    for col in train_data.columns:        if train_data[col].dtype == 'object':            le = LabelEncoder()            train_data[col] = le.fit_transform(train_data[col])    # Split the training data into training and validation sets    X = train_data.drop('Churn', axis=1)    # Impute missing values    imputer = SimpleImputer(strategy='mean')    X = imputer.fit_transform(X)    # Scale numerical features    scaler = StandardScaler()    X = scaler.fit_transform(X)    # Load the holdout data    holdout_data = pd.read_csv("../data/cell2cellholdout.csv")    # Store customer IDs    customer_ids = holdout_data['CustomerID']    # Remove customer_id    holdout_data = holdout_data.drop('CustomerID', axis=1)    # Remove Churn column if it exists    if 'Churn' in holdout_data.columns:        holdout_data = holdout_data.drop('Churn', axis=1)    # Handle missing values BEFORE Label Encoding    for col in holdout_data.columns:        if holdout_data[col].dtype == 'object':            holdout_data[col] = holdout_data[col].fillna('Unknown')    # Convert categorical features to numerical features    for col in holdout_data.columns:        if holdout_data[col].dtype == 'object':            le = LabelEncoder()            holdout_data[col] = le.fit_transform(holdout_data[col])    # Impute missing values    holdout_data = imputer.transform(holdout_data)    # Scale numerical features    holdout_data = scaler.transform(holdout_data)    # Load the model    filename = 'models/xgboost_model.pkl'    loaded_model = pickle.load(open(filename, 'rb'))    # Make predictions on the holdout set    y_pred_holdout = loaded_model.predict(holdout_data)    y_prob_holdout = loaded_model.predict_proba(holdout_data)[:, 1]    # Create a dataframe with customer IDs and predictions    predictions = pd.DataFrame({'CustomerID': customer_ids, 'predicted_churn': y_pred_holdout, 'churn_probability': y_prob_holdout})    # Save predictions to a CSV file in the dashboard/data directory    predictions.to_csv('../../Documents/Projects/telecom_churn_project/telecom_churn/dashboard/data/churn_predictions.csv', index=False)    print("Predictions saved to churn_predictions.csv")